In [41]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [42]:
train=pd.read_csv('edit_train.csv')
test=pd.read_csv('edit_test.csv')
submission=pd.read_csv('energy/sample_submission.csv', encoding='cp949')

In [43]:
columns = ['date_time']

In [44]:
train = train.drop(columns, axis=1)
test = test.drop(columns, axis=1)

In [45]:
features = train.drop('전력사용량(kWh)', axis=1)
label = train[['전력사용량(kWh)']]

In [46]:
kfold = KFold(n_splits=5, shuffle=True, random_state=0)
fold = 1
models = {}

for train_idx, valid_idx in kfold.split(train):
    print('\n ================== Fold {} =================='.format(fold))

    X_train, X_valid = features.iloc[train_idx, :], features.iloc[valid_idx, :]
    y_train, y_valid = label.iloc[train_idx, :], label.iloc[valid_idx, :]

    model = LGBMRegressor(n_estimators=10000, random_state=0)
    model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], early_stopping_rounds=30, verbose=50)
    models[fold] = model

    fold += 1


 ================== Fold 1 ==================
Training until validation scores don't improve for 30 rounds
[50]	training's l2: 145238	valid_1's l2: 150898
[100]	training's l2: 92675.7	valid_1's l2: 100451
[150]	training's l2: 77000.4	valid_1's l2: 85801.5
[200]	training's l2: 66975	valid_1's l2: 76924.5
[250]	training's l2: 60242.2	valid_1's l2: 71525.4
[300]	training's l2: 54646.6	valid_1's l2: 66956.9
[350]	training's l2: 50384.9	valid_1's l2: 63743.5
[400]	training's l2: 47726.6	valid_1's l2: 61949.1
[450]	training's l2: 44970.5	valid_1's l2: 59826.3
[500]	training's l2: 42717.4	valid_1's l2: 58605
[550]	training's l2: 40853.3	valid_1's l2: 57371.1
[600]	training's l2: 38970.9	valid_1's l2: 55730.5
[650]	training's l2: 37339.4	valid_1's l2: 54777.5
[700]	training's l2: 35985.5	valid_1's l2: 54030.2
[750]	training's l2: 34893.6	valid_1's l2: 53323
[800]	training's l2: 33793.8	valid_1's l2: 52653.3
[850]	training's l2: 32844.4	valid_1's l2: 52149.4
[900]	training's l2: 31980.4	valid_

In [47]:
submission = pd.read_csv('energy/sample_submission.csv', encoding='cp949')

for i in range(1, 6):
    submission['answer'] += models[i].predict(test)/5

submission.to_csv('lgbm_day1_3.csv', index=False)